# Rețele generative

Rețelele Neuronale Recurente (RNN) și variantele lor cu celule cu porți, cum ar fi Celulele cu Memorie pe Termen Lung (LSTM) și Unitățile Recurente cu Porți (GRU), au oferit un mecanism pentru modelarea limbajului, adică pot învăța ordonarea cuvintelor și pot oferi predicții pentru următorul cuvânt dintr-o secvență. Acest lucru ne permite să folosim RNN-urile pentru **sarcini generative**, cum ar fi generarea obișnuită de text, traducerea automată și chiar generarea de descrieri pentru imagini.

În arhitectura RNN discutată în unitatea anterioară, fiecare unitate RNN producea următoarea stare ascunsă ca ieșire. Totuși, putem adăuga și o altă ieșire fiecărei unități recurente, ceea ce ne-ar permite să generăm o **secvență** (care este egală ca lungime cu secvența originală). Mai mult, putem folosi unități RNN care nu acceptă o intrare la fiecare pas, ci doar primesc un vector de stare inițială și apoi produc o secvență de ieșiri.

În acest notebook, ne vom concentra pe modele generative simple care ne ajută să generăm text. Pentru simplitate, să construim o **rețea la nivel de caractere**, care generează text literă cu literă. În timpul antrenării, trebuie să luăm un corpus de text și să-l împărțim în secvențe de litere.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Construirea vocabularului de caractere

Pentru a construi o rețea generativă la nivel de caractere, trebuie să împărțim textul în caractere individuale, în loc de cuvinte. Stratul `TextVectorization` pe care l-am folosit anterior nu poate face acest lucru, așa că avem două opțiuni:

* Încărcarea manuală a textului și realizarea tokenizării „manual”, așa cum se arată în [acest exemplu oficial Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Utilizarea clasei `Tokenizer` pentru tokenizarea la nivel de caractere.

Vom alege a doua opțiune. `Tokenizer` poate fi folosit și pentru tokenizarea în cuvinte, astfel încât să fie ușor de trecut de la tokenizarea la nivel de caractere la cea la nivel de cuvinte.

Pentru a realiza tokenizarea la nivel de caractere, trebuie să transmitem parametrul `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Vrem să folosim și un token special pentru a indica **sfârșitul secvenței**, pe care îl vom numi `<eos>`. Să-l adăugăm manual la vocabular:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Antrenarea unui RNN generativ pentru a genera titluri

Modul în care vom antrena RNN pentru a genera titluri de știri este următorul. La fiecare pas, vom lua un titlu, care va fi introdus într-un RNN, iar pentru fiecare caracter de intrare vom cere rețelei să genereze următorul caracter de ieșire:

![Imagine care arată un exemplu de generare RNN a cuvântului 'HELLO'.](../../../../../translated_images/ro/rnn-generate.56c54afb52f9781d.webp)

Pentru ultimul caracter al secvenței noastre, vom cere rețelei să genereze token-ul `<eos>`.

Principala diferență între RNN-ul generativ pe care îl folosim aici este că vom lua o ieșire de la fiecare pas al RNN-ului, și nu doar de la celula finală. Acest lucru poate fi realizat prin specificarea parametrului `return_sequences` pentru celula RNN.

Astfel, în timpul antrenamentului, o intrare în rețea ar fi o secvență de caractere codificate de o anumită lungime, iar o ieșire ar fi o secvență de aceeași lungime, dar deplasată cu un element și terminată cu `<eos>`. Un minibatch va consta din mai multe astfel de secvențe, iar noi va trebui să folosim **padding** pentru a alinia toate secvențele.

Să creăm funcții care vor transforma setul de date pentru noi. Deoarece dorim să completăm secvențele la nivel de minibatch, mai întâi vom grupa setul de date apelând `.batch()`, iar apoi îl vom `map`-a pentru a face transformarea. Așadar, funcția de transformare va lua un întreg minibatch ca parametru:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Câteva lucruri importante pe care le facem aici:
* Mai întâi extragem textul efectiv din tensorul de tip string
* `text_to_sequences` convertește lista de șiruri de caractere într-o listă de tensori de tip întreg
* `pad_sequences` completează acești tensori la lungimea lor maximă
* În final, codificăm one-hot toți caracterii, realizăm și deplasarea, precum și adăugarea `<eos>`. Vom vedea în curând de ce avem nevoie de caractere codificate one-hot.

Totuși, această funcție este **Pythonică**, adică nu poate fi tradusă automat într-un grafic computațional Tensorflow. Vom primi erori dacă încercăm să folosim această funcție direct în funcția `Dataset.map`. Trebuie să închidem acest apel Pythonic utilizând wrapper-ul `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Note**: Diferențierea între funcțiile de transformare Pythonic și cele Tensorflow poate părea puțin prea complicată, și poate te întrebi de ce nu transformăm setul de date folosind funcții standard Python înainte de a-l transmite către `fit`. Deși acest lucru se poate face cu siguranță, utilizarea `Dataset.map` are un mare avantaj, deoarece pipeline-ul de transformare a datelor este executat folosind graful computațional Tensorflow, care profită de calculele GPU și minimizează necesitatea de a transfera date între CPU/GPU.

Acum putem construi rețeaua noastră generator și începe antrenarea. Aceasta poate fi bazată pe orice celulă recurentă pe care am discutat-o în unitatea anterioară (simplă, LSTM sau GRU). În exemplul nostru vom folosi LSTM.

Deoarece rețeaua primește caractere ca input, iar dimensiunea vocabularului este destul de mică, nu avem nevoie de un strat de embedding; input-ul codificat one-hot poate fi transmis direct în celula LSTM. Stratul de ieșire va fi un clasificator `Dense` care va converti ieșirea LSTM în numere de token-uri codificate one-hot.

În plus, deoarece lucrăm cu secvențe de lungime variabilă, putem folosi stratul `Masking` pentru a crea o mască care va ignora partea umplută a șirului. Acest lucru nu este strict necesar, deoarece nu suntem foarte interesați de tot ceea ce depășește token-ul `<eos>`, dar îl vom folosi pentru a câștiga ceva experiență cu acest tip de strat. `input_shape` va fi `(None, vocab_size)`, unde `None` indică secvența de lungime variabilă, iar forma de ieșire este `(None, vocab_size)` de asemenea, așa cum poți vedea din `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Generarea rezultatului

Acum că am antrenat modelul, vrem să-l folosim pentru a genera un rezultat. În primul rând, avem nevoie de o modalitate de a decoda textul reprezentat de o secvență de numere de tokeni. Pentru aceasta, am putea folosi funcția `tokenizer.sequences_to_texts`; totuși, aceasta nu funcționează bine cu tokenizarea la nivel de caracter. Prin urmare, vom lua un dicționar de tokeni din tokenizer (numit `word_index`), vom construi o hartă inversă și vom scrie propria noastră funcție de decodare:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Acum, să începem generarea. Vom începe cu un șir `start`, îl vom codifica într-o secvență `inp`, iar apoi, la fiecare pas, vom apela rețeaua noastră pentru a deduce următorul caracter.

Rezultatul rețelei `out` este un vector cu `vocab_size` elemente care reprezintă probabilitățile fiecărui token, iar noi putem găsi numărul tokenului cel mai probabil folosind `argmax`. Apoi, adăugăm acest caracter la lista generată de tokeni și continuăm procesul de generare. Acest proces de generare a unui caracter se repetă de `size` ori pentru a genera numărul necesar de caractere, și ne oprim mai devreme dacă întâlnim `eos_token`.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Eșantionarea rezultatelor în timpul antrenamentului

Deoarece nu avem metrici utile precum *acuratețea*, singura modalitate prin care putem observa că modelul nostru se îmbunătățește este prin **eșantionarea** șirurilor generate în timpul antrenamentului. Pentru a face acest lucru, vom folosi **callback-uri**, adică funcții pe care le putem transmite funcției `fit` și care vor fi apelate periodic în timpul antrenamentului.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Acest exemplu generează deja un text destul de bun, dar poate fi îmbunătățit în mai multe moduri:

* **Mai mult text**. Am folosit doar titluri pentru sarcina noastră, dar poate doriți să experimentați cu text complet. Rețineți că RNN-urile nu sunt foarte bune la gestionarea secvențelor lungi, așa că are sens fie să le împărțiți în propoziții mai scurte, fie să antrenați întotdeauna pe o lungime fixă de secvență cu o valoare predefinită `num_chars` (de exemplu, 256). Puteți încerca să modificați exemplul de mai sus într-o astfel de arhitectură, folosind [tutorialul oficial Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) ca inspirație.

* **LSTM cu mai multe straturi**. Este logic să încercați 2 sau 3 straturi de celule LSTM. Așa cum am menționat în unitatea anterioară, fiecare strat de LSTM extrage anumite modele din text, iar în cazul generatorului la nivel de caractere, ne putem aștepta ca nivelul inferior al LSTM să fie responsabil pentru extragerea silabelor, iar nivelurile superioare - pentru cuvinte și combinații de cuvinte. Acest lucru poate fi implementat simplu prin transmiterea unui parametru pentru numărul de straturi către constructorul LSTM.

* De asemenea, puteți experimenta cu **unități GRU** și să vedeți care performează mai bine, precum și cu **dimensiuni diferite ale straturilor ascunse**. O dimensiune prea mare a stratului ascuns poate duce la supraînvățare (de exemplu, rețeaua va învăța textul exact), iar o dimensiune mai mică s-ar putea să nu producă rezultate bune.


## Generarea textului soft și temperatura

În definiția anterioară a funcției `generate`, alegeam întotdeauna caracterul cu cea mai mare probabilitate ca următor caracter în textul generat. Acest lucru ducea adesea la faptul că textul "cicla" între aceleași secvențe de caractere, din nou și din nou, ca în acest exemplu:
```
today of the second the company and a second the company ...
```

Totuși, dacă analizăm distribuția probabilităților pentru următorul caracter, este posibil ca diferența dintre câteva dintre cele mai mari probabilități să nu fie semnificativă, de exemplu, un caracter poate avea probabilitatea 0.2, iar altul - 0.19, etc. De exemplu, când căutăm următorul caracter în secvența '*play*', următorul caracter poate fi la fel de bine fie un spațiu, fie **e** (ca în cuvântul *player*).

Acest lucru ne conduce la concluzia că nu este întotdeauna "corect" să selectăm caracterul cu probabilitatea cea mai mare, deoarece alegerea celui de-al doilea cel mai probabil caracter poate duce tot la un text semnificativ. Este mai înțelept să **eșantionăm** caracterele din distribuția de probabilitate oferită de rezultatul rețelei.

Această eșantionare poate fi realizată folosind funcția `np.multinomial`, care implementează așa-numita **distribuție multinomială**. O funcție care implementează această generare de text **soft** este definită mai jos:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Am introdus încă un parametru numit **temperatură**, care este utilizat pentru a indica cât de strict ar trebui să ne bazăm pe cea mai mare probabilitate. Dacă temperatura este 1.0, facem o eșantionare multinomială corectă, iar când temperatura merge spre infinit - toate probabilitățile devin egale și selectăm aleator următorul caracter. În exemplul de mai jos putem observa că textul devine lipsit de sens atunci când creștem temperatura prea mult și seamănă cu un text "ciclic" generat rigid atunci când se apropie de 0.



---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să fiți conștienți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
